## Calculate Bering Strait Transport


## Import packages

In [1]:
%matplotlib inline
import numpy as np
import xarray as xr
import s3fs
import zarr
import util
import warnings
warnings.filterwarnings('ignore')

## Create a new Dask cluster with the Dask Gateway

In [2]:
from dask_gateway import Gateway
gateway = Gateway()

In [ ]:
##A line of trick to clean your dask cluster before you start your computation
from dask.distributed import Client
clusters=gateway.list_clusters()
print(clusters )
for cluster in clusters :
    cluster= gateway.connect(cluster.name)
    print(cluster)
    client = Client(cluster)
    client.close()
    cluster.shutdown()

In [4]:
cluster = gateway.new_cluster(worker_memory=2, worker_cores=1)

cluster.scale(8)
cluster

## Get a client from the Dask Gateway Cluster

As stated above, creating a Dask `Client` is mandatory in order to perform following Daks computations on your Dask Cluster.

In [5]:
from distributed import Client

if cluster:
    client = Client(cluster) # create a dask Gateway cluster
else:
    client = Client()   # create a local dask cluster on the machine.
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /jupyterhub/services/dask-gateway/clusters/daskhub.80064203c68c4960bf4a6cc3ea1dc95c/status,


## Open dictionary of OMIP2 simulations

In [6]:
# Load in file of model names and fnames
model_fnames_dict = np.load("models.npy", allow_pickle=True).item()

In [7]:
list(model_fnames_dict.keys())

['CNRM-CM6-1-HR',
 'MRI-ESM2-0',
 'CMCC-CM2-SR5',
 'ACCESS-OM2',
 'TaiESM1-TIMCOM2',
 'MIROC6',
 'ACCESS-OM2-025',
 'CMCC-CM2-HR4',
 'NorESM2-LM',
 'EC-Earth3',
 'CNRM-CM6-1',
 'FGOALS-f3-L',
 'TaiESM1-TIMCOM']

## Function for reading data from zarr

In [3]:
def read_dataset_from_zarr(sim_name):
    # set path on bucket
    path='WAFFLES/OMIP2'
    s3_prefix =  "s3://"+path
    print(s3_prefix)
    
    # get storage keys
    access_key = !aws configure get aws_access_key_id
    access_key = access_key[0]
    secret_key = !aws configure get aws_secret_access_key
    secret_key = secret_key[0]
    
    # set storage target
    client_kwargs={'endpoint_url': 'https://object-store.cloud.muni.cz'}
    target = s3fs.S3FileSystem(anon=False,client_kwargs=client_kwargs)
    
    # file name for variable in simulation
    zarr_file_name= sim_name
    uri = f"{s3_prefix}/{zarr_file_name}"
    
    # get store argument for zarr
    store = zarr.storage.FSStore(uri,client_kwargs=client_kwargs,
                                 key=access_key, secret=secret_key)
    
    # read variable from zarr
    ds=xr.open_zarr(store=store)
    
    return ds

## Read, subset, and calc transports

In [2]:
## Set Bering Straight region 
lon_bs_min = 190
lon_bs_max = 192
#Note that most models are very sensitive to the maxlon value as most of the grids slant that way
lat_bs = 66 + (19.6/60) - 0.65

In [ ]:
for sim_name in list(model_fnames_dict.keys()):
    # read in ds from bucket
    ds = read_dataset_from_zarr(sim_name)
    
    # extract Bering Strait transect
    ds_BS = util.subset_ds_bering_trans(ds,sim_name,lat_bs,lon_bs_min,lon_bs_max)
    
    # Calc transports
    ds_BS = util.calc_Bering_fluxes(ds_BS)
    
    # write transport flux files?
    

## Test code below this line